# Metadata Refresh for SQL Server (Project Setup)

This notebook contains the SQL Set-up scripts for the eltSnap project "Metadata Refresh for SQL Server"

> To run these SQL scripts, first connect to your SQL Server instance of **eltsnap\_v2**

## Add a Connection for the SQL Server where you want to Extract Metadata (Source)

Verify the Parameter Values in the EXEC statement below. **Note:** You will **first** want to _replace_ 'localhost' with your _server_name_ and 'master' with your _target database_

> Note: Initial Parameter Values can be substituted at runtime

In [ ]:
IF NOT EXISTS (SELECT 1 FROM [elt].[oledb_connection] WHERE [connection_name] = 'sql_server')
  EXEC [elt].[Save Connection by Name] 
      @connection_name = 'sql_server'
    , @server_name = 'localhost'
    , @database_name = 'master'
    , @provider = 'SQL Server Native Client 11.0'

## Check Impact of Importing "Notebook Based" Project (Optional)

In [ ]:
EXEC [ads].[Impact Check from ADS] 'Metadata Refresh v2', 'notebook_snap_basics\content\metadata_refresh_for_sql_server_project.ipynb';

## Import the "Notebook Based" Project into eltSnap

The following script is used to Import the project

In [ ]:
EXEC [ads].[Import Project from ADS] 'Metadata Refresh v2';

## Prevent Job from Running Concurrently (Recommended)

To prevent this job from running concurrently (_including other related Metadata Refresh jobs_), the following script can be run to apply the needed 'runtime' constraint

In [ ]:
DECLARE @project_name_match NVARCHAR(128) = 'Metadata Refresh%'
	  , @rule NVARCHAR(128) = 'Cannot Run While Any'
	  , @rule_qualifier NVARCHAR(128) = 'Is Running';

/* Delete prior constraints if exists */
DELETE [elt].[project_rule_constraints]
 WHERE [project_id] IN 
     ( SELECT [project_id]
	     FROM [elt].[project]
        WHERE [project_name] LIKE @project_name_match
     )
   AND [rule_id] =
     ( SELECT [rule_id]
	     FROM [elt].[project_rule]
		WHERE [rule] = @rule AND [rule_qualifier] = @rule_qualifier
	 );

/* Add constraints to prevent concurrent runs */
WITH [prj_id_grp] AS
(
SELECT (SELECT STRING_AGG([project_id], ',') WITHIN GROUP (ORDER BY [project_id])) AS [referenced_projects]
  FROM [elt].[project]
 WHERE [project_name] LIKE @project_name_match
)
INSERT INTO [elt].[project_rule_constraints]
SELECT NEWID() AS [id]
     , 1 AS [wait_time_quantity]
	 , 'Minutes' AS [wait_time_unit]
	 , [project_id]
	 , (SELECT [rule_id] FROM [elt].[project_rule] WHERE [rule] = @rule AND [rule_qualifier] = @rule_qualifier) AS [rule_id]
	 , [referenced_projects]
	 , NULL AS [rule_quantity]
	 , NULL AS [rule_date_part]
  FROM [elt].[project]
     , [prj_id_grp]
 WHERE [project_name] LIKE @project_name_match;